# Scraping Medienlandschaft Deutschlands

Dieses Jupyter Notebook beinhaltet das Scraping der Medienlandschaft Deutschlands, wie sie im eigens für die Masterarbeit angefertigten Datensatz abgebildet wurde.

In [1]:
# basis packages
import requests
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
from pprint import pprint
import math
import json
from tqdm import tqdm, trange
from time import sleep

# scrape packages: snscrape
import snscrape.modules.twitter as sntwitter
import itertools

## Schritt 1: Laden des Datensatzes

Zunächst wird der Datensatz als Pandas Dataframe geladen. Dazu sind nur die Sheets des ODS Dokuments ab Sheet 2 notwendig.

Beachten bei Laden von ODS-Dokumenten: `odfpy` muss als engine verfügbar sein. Dann einfach mit `pd.read_excel` einlesen.

In [2]:
ueberreg = pd.read_excel("/home/fw/Documents/uni/masterarbeit/scraping/datensätze/sammlung_medien.ods",
              sheet_name=1,
              engine="odf")

regio = pd.read_excel("/home/fw/Documents/uni/masterarbeit/scraping/datensätze/sammlung_medien.ods",
              sheet_name=2,
              engine="odf")

orr = pd.read_excel("/home/fw/Documents/uni/masterarbeit/scraping/datensätze/sammlung_medien.ods",
              sheet_name=3,
              engine="odf")
digi = pd.read_excel("/home/fw/Documents/uni/masterarbeit/scraping/datensätze/sammlung_medien.ods",
              sheet_name=4,
              engine="odf")

sammlung = pd.read_excel("/home/fw/Documents/uni/masterarbeit/scraping/datensätze/sammlung_medien.ods",
              sheet_name=5,
              engine="odf")

In [3]:
sammlung

,Name,Twitter,Klassifizierung,Verbreitungsart,Gebiet,InhaltThema,Herkunft,RVerkauf,RLeser,RVisits,Erreichbarkeit,NormenWerte,Geschäftsmodell,Anmerkungen,Quellen,Einordnung,Infos
0,ARD Morgenmagazin,ardmoma,TV/Radio,Offline,überregional,Nachrichten,NaN,NaN,NaN,NaN,1.0,1,1,aktiv,NaN,NaN,NaN
1,ARD Presseclub,ARD_Presseclub,TV/Radio,Offline,überregional,Nachrichten,NaN,NaN,NaN,NaN,1.0,1,1,aktiv,NaN,NaN,NaN
2,ARD tagesschau,tagesschau,TV/Radio,Offline,überregional,Nachrichten,NaN,NaN,NaN,117600000.0,1.0,1,1,aktiv,NaN,NaN,NaN
3,ARD tagesschau EIL,tagesschau_eil,TV/Radio,Twitter,überregional,Eilmeldungen,NaN,NaN,NaN,NaN,1.0,1,1,aktiv,NaN,NaN,NaN
4,ARD tagesschau24,tagesschau24,TV/Radio,Offline,überregional,Nachrichten,NaN,NaN,NaN,NaN,1.0,1,1,aktiv,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,Übermedien,uebermedien,Zeitschrift,Offline+Online,überregional,Meinungen,NaN,NaN,NaN,503800.0,3.0,2,3,aktiv,NaN,aufklärerisch-informativ,Magazin für Medienkritik
544,upday DE,updayDE,Nachrichtenportal,Online,überregional,Nachrichten,NaN,NaN,NaN,24500000.0,1.0,3,2,leicht veraltet,NaN,Aggregator,NaN
545,Verfassungsblog,Verfassungsblog,Blog,Online,überregional,Politik,NaN,NaN,NaN,337600.0,4.0,4,3,aktiv,NaN,fachspezifisches Blog,"über Liste, Faktenkontor: beliebte dt. Blogs (..."
546,Volksverpetzer,Volksverpetzer,Blog,Online,überregional,Politik,NaN,NaN,NaN,456800.0,3.0,2,3,aktiv,NaN,aufklärerisch-informativ,Recherche; Faktenchecks


In [4]:
twlist = list(sammlung["Twitter"])
twitternamen_medien = [item for item in twlist if not(pd.isnull(item)) == True]
twitternamen_medien

['ardmoma',
 'ARD_Presseclub',
 'tagesschau',
 'tagesschau_eil',
 'tagesschau24',
 'tagesthemen',
 'BILD',
 'BILD_News',
 'BILD_Politik',
 'dpa',
 'focusonline',
 'FOCUS_TopNews',
 'faznet',
 'FAZ_Eil',
 'fr',
 'jungewelt',
 'ntvde',
 'ntv_EIL',
 'NZZde',
 'NZZ',
 'phoenix_de',
 'reuters_de',
 'RND_de',
 'rtl_aktuell',
 'SPIEGEL_EIL',
 'SPIEGEL_Top',
 'derspiegel',
 'sternde',
 'SZ',
 'SZ_TopNews',
 'tazgezwitscher',
 'taz_news',
 'welt',
 'WELTAMSONNTAG',
 'ZDFheute',
 'heutejournal',
 'DIEZEIT',
 'zeitonline',
 'ab_nachrichten',
 'bkz_online',
 'BNN_BaNeuNa',
 'badischezeitung',
 'bzonline',
 'kreiszeitungbb',
 'ez_online',
 'Gaeubote',
 'GT_Gmuend',
 'goodnews_stgt',
 'Pressehaus',
 'stimmeonline',
 'hz_nachrichten',
 'kanews',
 'konradsblatt',
 'KontextWZ',
 'LKZ_Leonberg',
 'LKZ_online',
 'mainecho_de',
 'online_MM',
 'MZ_nachrichten',
 'mittelbadische',
 'TagblattOnline',
 'pznews',
 'remszeitung',
 'geaonline',
 'rn_nachrichten',
 'RNZonline',
 'SchwaePo',
 'Schwaebische',
 'Tag

In [5]:
medienliste = ['ardmoma', 'ARD_Presseclub', 'tagesschau', 'tagesschau_eil', 'tagesschau24',
               'tagesthemen', 'BILD', 'BILD_News', 'BILD_Politik', 'dpa','focusonline', 'FOCUS_TopNews',
               'faznet', 'FAZ_Eil', 'fr', 'jungewelt', 'ntvde', 'ntv_EIL', 'NZZde', 'NZZ', 'phoenix_de',
               'reuters_de', 'RND_de', 'rtl_aktuell', 'SPIEGEL_EIL', 'SPIEGEL_Top', 'derspiegel',
               'sternde', 'SZ', 'SZ_TopNews', 'tazgezwitscher', 'taz_news', 'welt', 'WELTAMSONNTAG',
               'ZDFheute', 'heutejournal', 'DIEZEIT', 'zeitonline', 'ab_nachrichten', 'bkz_online',
               'BNN_BaNeuNa', 'badischezeitung', 'bzonline', 'kreiszeitungbb', 'ez_online', 'Gaeubote',
               'GT_Gmuend', 'goodnews_stgt', 'Pressehaus', 'stimmeonline', 'hz_nachrichten', 'kanews',
               'konradsblatt', 'KontextWZ', 'LKZ_Leonberg', 'LKZ_online', 'mainecho_de', 'online_MM',
               'MZ_nachrichten', 'mittelbadische', 'TagblattOnline', 'pznews', 'remszeitung',
               'geaonline', 'rn_nachrichten', 'RNZonline', 'SchwaePo', 'Schwaebische', 'Tagblatt',
               'schwarzwaelder', 'KN_Wochenblatt', 'StN_News', 'StZ_NEWS', 'Suedkurier_News', 'SWPde',
               'Der_Teckbote', 'wnoz', 'zvw_redaktion', 'ZAK_Redaktion', 'Abendzeitung', 'AZ_Allgaeu',
               'AZ_Augsburg', 'GZ_AKTUELL', 'BayerRundschau', 'Staatszeitung', 'bt_tagblatt',
               'CTageblatt', 'donaukurier', 'ed_anzeiger', 'EuroamSonntag', 'Frankenpost',
               'fraenkischertag', 'GermaniaPlus', 'pnp_grafenau', 'hallo_muenchen', 'heinrichsblatt',
               'HZOnline', 'KreisboteOA', 'mainpost', 'MarktSpiegel', 'idowa', 'MZ_Online', 'mz_de',
               'merkur_de', 'Wochenanzeiger', 'NPCoburg', 'onetz_de', 'kurier_online', 'NN_Online',
               'NZ_Online', 'heimatzeitungen', 'ObermainTB', 'pnp', 'Pegnitz_Zeitung', 'pnp_plattling',
               'SaaleZeitung', 'sonntagsblatt', 'StaZ_Augsburg', 'SuddtZeitung', 'tzmuenchen',
               'wochenblattnews', 'BerlAbendblatt', 'BERLINER_KURIER', 'morgenpost', 'berlinerwoche',
               'berlinerzeitung', 'bzberlin', 'JuedischeOnline', 'Junge_Freiheit', 'Jungle_World',
               'ndaktuell', 'pukzeitung', 'tag_des_herrn', 'Tagesspiegel', 'TspCheckpoint', 'vorwaerts',
               'lr_online', 'maz_online', 'mozde', 'NCasnik', 'PNN_de', 'nordseezeitung',
               'weserkurier', 'Wuemme_Zeitung', 'bgzonline', 'abendblatt', 'mopo', 'paz_ob',
               'Kirchenzeitung', 'boersenzeitung', 'mittelhessende', 'Echo_Online', 'fnp_zeitung',
               'fuldaerzeitung', 'ANZEIGER_NEWS', 'MitmachZeitung', 'HA1725', 'HNA_online',
               'KAgezwitscher', 'Laredaktion', 'mspnachrichten', 'opmarburg', 'oberhessische',
               'Echo_Online', 'Zwitschern_UA', 'Werra_Rundschau', 'WZ_Wetterau', 'wknachrichten',
               'Nordkurier', 'OZlive', 'azgifhorn', 'Harlinger1862', 'BZRedaktion', 'BorkumerZeitung',
               'BS_Zeitung', 'cezett', 'CN_Online', 'Dewezet', 'Kreisblatt', 'einmorgenpost',
               'ejzgezwitscher', 'EmderZeitung', 'ga_online', 'goslarsche', 'goetageblatt',
               'GN_Nordhorn', 'HAZ', 'Die_Harke', 'HarzKurier', 'hinews', 'Wochenblatt1791',
               'Kreiszeitung', 'kzw_redaktion', 'lzgezwitscher', 'neuepresse', 'noz_de',
               'NEZ_Online', 'nwzonline', 'oz_online_de', 'on_online_de', 'pazpeine', 'RZ_Rheiderland',
               'Rundblick_NDS', 'sn_online', 'SZLZ1', 'TAGEBLATTonline', 'wz_net', 'WZonline',
               'wazwolfsburg', 'aachenerzeitung', 'BA_Billerbeck', 'blickaktuell', 'BBV_Bocholt',
               'BorkenerZeitung', 'DZ_Duelmen', 'gabonn', 'DieGlocke', 'handwerksblatt', 'express24',
               'KreisblattHalle', 'handelsblatt', 'HellwegerNews', 'ivz_aktuell', 'KevelaererBlatt',
               'KSTA', 'KoelnischeR', 'lzonline', 'MT_Online', 'MZ_MUENSTER', 'nwnews',
               'rgaonline', 'rponline', 'RN_DORTMUND', 'Sauerlandkurier', 'SiegenerZeitung',
               'soesteranzeiger', 'SGTageblatt', 'UnsereKirche', 'UnsereZeit_UZ', 'WAZ_Redaktion',
               'wznewsline', 'westfalenblatt', 'DasSauerland', 'WN_Redaktion', 'WA_online',
               'WRundschau', 'aznachrichten', 'PflzischerMerkur', 'pz_online', 'rheinpfalz',
               'RheinZeitung', 'STagblatt', 'Volksfreund', 'wznachrichten', 'szaktuell', 'wo_regio_sb',
               'dnn_online', 'freie_presse', 'LIZ_de', 'LVZ', 'saechsischeDE', 'sonntagticker',
               'TorgauerZeitung', 'AltmarkZeitung', 'glaubeundheimat', 'mzwebde', 'Volksstimme',
               'volksstimme_md', 'BassesBlatt', 'BoyensMedien', 'Flensborg_Avis', 'FTageblatt',
               'kn_online', 'LN_Online', 'der_reporter', 'shz_de', 'mywochenspiegel',
               'freieswort', 'OTZonline', 'TAOnline', 'TLZnews', 'ARTEde', 'BR_Presse', 'BR24',
               'DeutscheWelle', 'DLF', 'dlfnova', 'dlfkultur', 'hessenschau', 'hrPresse', 'mdrde',
               'MDRAktuell', 'MDRpresse', 'MDR_SN', 'MDR_SAN', 'MDR_th', 'NDRinfo', 'NDRnds',
               'NDRsh', 'ndr', 'butenunbinnen', 'rbbabendschau', 'rbb24', 'rbb24Inforadio',
               'SRKommunikation', 'SRaktuell', 'SWRAktuellBW', 'SWRpresse', 'SWRAktuellRP', 'WDR',
               'WDR2', 'WDRaktuell', 'a_watch', 'Achgut_com', 'jreichelt', 'acTVismMunich',
               'apolut_net', 'cicero_online', 'langemannmedien', 'comeon_de', 'COMPACTMagazin',
               'correctiv_org', 'derfreitag', 'Weltwoche', 'GMX', 'HalleSpektrum', 'Hintergrund_de',
               'infosperber', 'inFranken', 'JungNaiv', 'stv_klartext', 'krassundkonkret', 'krautreporter',
               'LRM_dietalkshow', 'lobbycontrol', 'mucbook', 'NachDenkSeiten', 'NNNonline', 'nordbayern',
               'NuoVisoTV', 'OvertonMagazin', 'pressenza_ger', 'reitschuster', 'rt_deutsch',
               'ruhrbarone', 'ServusTV_News', 'svz_de', 'tonline', 'TAG24', 'talkimhangar7',
               'telepolis_news', 'theeuropean', 'TichysEinblick', 'uebermedien', 'updayDE',
               'Verfassungsblog', 'Volksverpetzer', 'WEBDENews']

In [6]:
## extra zelle, damit nicht überschrieben pro scrape
tweets_medien = []

In [10]:
# namen immer einzeln neu einsetzen, da sonst fehler mit neuer Twitter API
namen_medien = ['kn_online', 'LN_Online', 'der_reporter', 'shz_de', 'mywochenspiegel',
               'freieswort', 'OTZonline', 'TAOnline', 'TLZnews']
maximum = 25000
#pbar = tqdm(total=counter)

# jedes mal überschreiben bitte!
tweets_zwischen = []

for n, k in enumerate(namen_medien):
    # tqdm hier: anzeige der bisher durchgeführten iterationen des codes;
    # total=maximum+1 verwandelt das in progress bar mit 15001=100%
    for i, tweet in enumerate(tqdm(sntwitter.TwitterSearchScraper('from:{} since:2022-01-01 until:2022-11-01'.format(namen_medien[n])).get_items(), total=maximum)):
        
        # break bedingung bei maximum an tweets/person festsetzen (evtl. ratsam bei riesigen mengen)
        #if i>maximum:
        #    break
        
        # für die anzahl tweets pro einzelscraping
        tweets_zwischen.append([tweet.date, tweet.id, tweet.rawContent,
                                tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount,
                                tweet.media, tweet.retweetedTweet, tweet.quotedTweet,
                                tweet.inReplyToTweetId, tweet.inReplyToUser, tweet.mentionedUsers,
                                tweet.coordinates, tweet.place, tweet.hashtags, tweet.cashtags,
                                tweet.user.username, tweet.user.id, tweet.user.location,
                                tweet.user.followersCount])
        tweets_zwischenlauf = pd.DataFrame(tweets_zwischen, columns = ["dateTime", "tweetID", "content",
                                                            "replies", "retweets", "likes", "quotes",
                                                            "media", "isRetweeted", "isQuoted",
                                                            "repliedTo_ID", "repliedTo_user", "mentionedUsers",
                                                            "coords", "tweetLoc", "hashtagList", "cashtags",
                                                            "user", "userID", "userLoc", "followerAmount"])
        
        # welche attribute für gesamtdaten abgegriffen werden
        tweets_medien.append([tweet.date, tweet.id, tweet.rawContent,
                                tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount,
                                tweet.media, tweet.retweetedTweet, tweet.quotedTweet,
                                tweet.inReplyToTweetId, tweet.inReplyToUser, tweet.mentionedUsers,
                                tweet.coordinates, tweet.place, tweet.hashtags, tweet.cashtags,
                                tweet.user.username, tweet.user.id, tweet.user.location,
                                tweet.user.followersCount])

        # gleich in dataframe umwandeln
        medientweets = pd.DataFrame(tweets_medien, columns = ["dateTime", "tweetID", "content",
                                                            "replies", "retweets", "likes", "quotes",
                                                            "media", "isRetweeted", "isQuoted",
                                                            "repliedTo_ID", "repliedTo_user", "mentionedUsers",
                                                            "coords", "tweetLoc", "hashtagList", "cashtags",
                                                            "user", "userID", "userLoc", "followerAmount"])
        
# ausgabe zusätzlicher statistik am ende des scrapings: anz. insgesamter tweets nach scraping
# + tweets des scraping durchgangs
num_tweets_scrape = tweets_zwischenlauf.shape[0]
num_tweets_all = medientweets.shape[0]
#print(counter)
print(f'Anzahl der Tweets in diesem Durchgang: {num_tweets_scrape} Tweets.')
print(f'Insgesamt befinden sich im Datensatz: {num_tweets_all} Tweets')

 22%|████████                            | 5576/25000 [33:11<1:55:36,  2.80it/s]

Anzahl der Tweets in diesem Durchgang: 32523 Tweets.
Insgesamt befinden sich im Datensatz: 64660 Tweets


In [11]:
medientweets

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,repliedTo_user,mentionedUsers,coords,tweetLoc,hashtagList,cashtags,user,userID,userLoc,followerAmount
0,2022-10-07 06:23:03+00:00,1578269654992068608,Uns interessiert eure Meinung. Jetzt an der Le...,0,1,1,0,None,None,None,...,None,None,None,None,None,None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
1,2022-09-16 11:53:07+00:00,1570742573949669376,Ein tönendes Erz oder eine klingende Schelle -...,0,1,1,0,None,None,None,...,None,None,None,None,None,None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
2,2022-08-12 11:07:24+00:00,1558047492687728640,#freitagvoreins - unsere Seite 1. In der Kolum...,0,1,1,0,None,None,None,...,None,None,None,None,[freitagvoreins],None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
3,2022-07-12 15:28:45+00:00,1546879242717241344,Jetzt spricht der Bräutigam https://t.co/C3yjx...,1,2,1,0,None,None,None,...,None,None,None,None,None,None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
4,2022-07-06 09:06:15+00:00,1544608654271418368,Triegel trifft Cranach. Denkmalpflege trifft D...,0,2,2,0,None,None,None,...,None,None,None,None,None,None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64655,2022-01-01 15:05:19+00:00,1477294872587550720,Ein Feuerwerk an der Gedenkstätte #Buchenwald ...,0,0,0,0,None,None,None,...,None,None,None,None,[Buchenwald],None,TLZnews,21937777,Weimar / Germany,7199
64656,2022-01-01 14:10:14+00:00,1477281011226619904,Thüringens Wirtschaftsminister Tiefensee sieht...,1,0,0,1,None,None,None,...,None,None,None,None,None,None,TLZnews,21937777,Weimar / Germany,7199
64657,2022-01-01 11:40:38+00:00,1477243363242000386,Thüringen bleibt Bundesland mit höchster Coron...,0,0,0,0,None,None,None,...,None,None,None,None,None,None,TLZnews,21937777,Weimar / Germany,7199
64658,2022-01-01 10:46:02+00:00,1477229621854756864,Ruhestand: Rente: Das droht jedem dritten Voll...,0,0,0,0,None,None,None,...,None,None,None,None,None,None,TLZnews,21937777,Weimar / Germany,7199


In [12]:
sicherung_medien = medientweets
sicherung_medien.to_csv("sicherung_medien_regio_abGlaubeUndHeimat.csv", encoding='utf-8')
sicherung_medien

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,repliedTo_user,mentionedUsers,coords,tweetLoc,hashtagList,cashtags,user,userID,userLoc,followerAmount
0,2022-10-07 06:23:03+00:00,1578269654992068608,Uns interessiert eure Meinung. Jetzt an der Le...,0,1,1,0,None,None,None,...,None,None,None,None,None,None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
1,2022-09-16 11:53:07+00:00,1570742573949669376,Ein tönendes Erz oder eine klingende Schelle -...,0,1,1,0,None,None,None,...,None,None,None,None,None,None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
2,2022-08-12 11:07:24+00:00,1558047492687728640,#freitagvoreins - unsere Seite 1. In der Kolum...,0,1,1,0,None,None,None,...,None,None,None,None,[freitagvoreins],None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
3,2022-07-12 15:28:45+00:00,1546879242717241344,Jetzt spricht der Bräutigam https://t.co/C3yjx...,1,2,1,0,None,None,None,...,None,None,None,None,None,None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
4,2022-07-06 09:06:15+00:00,1544608654271418368,Triegel trifft Cranach. Denkmalpflege trifft D...,0,2,2,0,None,None,None,...,None,None,None,None,None,None,glaubeundheimat,963795284731023362,"Weimar, Deutschland",189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64655,2022-01-01 15:05:19+00:00,1477294872587550720,Ein Feuerwerk an der Gedenkstätte #Buchenwald ...,0,0,0,0,None,None,None,...,None,None,None,None,[Buchenwald],None,TLZnews,21937777,Weimar / Germany,7199
64656,2022-01-01 14:10:14+00:00,1477281011226619904,Thüringens Wirtschaftsminister Tiefensee sieht...,1,0,0,1,None,None,None,...,None,None,None,None,None,None,TLZnews,21937777,Weimar / Germany,7199
64657,2022-01-01 11:40:38+00:00,1477243363242000386,Thüringen bleibt Bundesland mit höchster Coron...,0,0,0,0,None,None,None,...,None,None,None,None,None,None,TLZnews,21937777,Weimar / Germany,7199
64658,2022-01-01 10:46:02+00:00,1477229621854756864,Ruhestand: Rente: Das droht jedem dritten Voll...,0,0,0,0,None,None,None,...,None,None,None,None,None,None,TLZnews,21937777,Weimar / Germany,7199


In [34]:
#sicherung_medien.to_csv("sicherung_medien_bisPhoenix.csv", encoding='utf-8')